---
title: "强化学习专题 - 时序差分方法"
date: 2025-06-06T23:22:00+08:00
author: "Liu Zheng"
tags: ["笔记", "实验", "AI", "强化学习"]
categories: "实验笔记"
---

# 时序差分方法

时序差分（TD）学习是强化学习中的一类核心方法，它结合了蒙特卡洛（MC）方法和动态规划（DP）的思想 。时序差分方法能够在不了解环境完整模型的情况下，通过从经验中学习来更新价值函数 。

## 数学原理